In [66]:
import math
import numpy as np
import pandas as pd
import scipy.stats as stats


buy option sell the stock. Realised vol comes in higher than implied vol

In [476]:

# Value of a call option:
def N(x):
    return stats.norm.cdf(x, loc=0, scale=1)



#T must be given in years

def BS(K,S,r,implied_vol,T):
    d1 = (math.log(S/K)+(r+(implied_vol**2)/2)*T)/(implied_vol*(np.sqrt(T)))
    d2 = d1 - np.sqrt(T) * implied_vol

    c = S*N(d1)-K*math.exp(-r*T)*N(d2)
    return c

Num = 100

day = pd.Series(range(Num,0,-1),name = 'Days till expiry')
K = 100
r = 0.0
realised_vol = 2
implied_vol = 1
mean = 100

stock_price = np.random.normal(mean,realised_vol,len(day)+1)
last_s_price = stock_price.tolist().pop()
stock_price = pd.Series(stock_price[:-1],index = day.index,name = 'Stock Price')

df = pd.concat([day,stock_price],axis = 1)

def delta(K, r, implied_vol, S, T):
    T = T/365
    d1 = (math.log(S/K)+(r+(implied_vol**2)/2)*T)/(implied_vol*(np.sqrt(T)))
    delta = N(d1)
    return delta
    # return d1



df['Delta'] = df.apply(lambda row: delta(K, r, implied_vol, row['Stock Price'], row['Days till expiry']), axis=1)

# Calculate "Number of shares to buy" column
df['Number of shares to buy'] = 0  # Initialize the column to zero

# Calculate the "Number of shares to buy" column iteratively
new_delta = 0
for i,j in df.iterrows():
    # print(j)
    # print(total_delta)
    delta = j['Delta']
    shares_to_buy = delta - new_delta
    new_delta += shares_to_buy

    
    df.at[i, 'Number of shares to buy'] = -shares_to_buy

# df['Option value'] = df.apply(lambda row: BS(K,row['Stock Price'],r,implied_vol,row['Days till expiry']),axis =1)
# df['Portfolio value'] = df['Option value']+df['Number of shares to buy'].cumsum()*df['Stock Price']
df['cost of shares'] = df['Number of shares to buy']*df['Stock Price']
df['Net stock position'] = df['Number of shares to buy'].cumsum()
# df['Cost of hedging'] = df['cost of shares'].cumsum()

# last_row = pd.Series(data = [0,101,0,-1*df['Net stock position'][-1],-1*df['Net stock position'][-1]*df['Stock Price'][-1],0])
# df
df.loc[len(df)] = [0,last_s_price,0,-df['Net stock position'].iloc[-1],-df['Net stock position'].iloc[-1]*df['Stock Price'].iloc[-1],0]

In [477]:
option_cost = BS(K,df['Stock Price'].iloc[0],r,implied_vol,df['Days till expiry'].iloc[0]/365)
option_cost

option_pnl = (max((df['Stock Price'].iloc[-1]-K),0) - option_cost)
option_pnl

stock_pnl = -df['cost of shares'].sum()
stock_pnl

PnL = stock_pnl+option_pnl
PnL

print('Option Cost: {}'.format(option_cost))
print('Option PnL: {}, Stock PnL: {}, TOTAL PnL: {}'.format(option_pnl,stock_pnl,PnL))

Option Cost: 18.856953404596545
Option PnL: -16.48859163736173, Stock PnL: 0.5761502490686468, TOTAL PnL: -15.912441388293082


In [483]:
def get_dynamically_hedged_PnL(realised_vol,implied_vol):
    Num = 100

    day = pd.Series(range(Num,0,-1),name = 'Days till expiry')
    K = 100
    r = 0.0
    # realised_vol = 2
    # implied_vol = 1
    mean = 100

    stock_price = np.random.normal(mean,realised_vol,len(day)+1)
    last_s_price = stock_price.tolist().pop()
    stock_price = pd.Series(stock_price[:-1],index = day.index,name = 'Stock Price')

    df = pd.concat([day,stock_price],axis = 1)

    def delta(K, r, implied_vol, S, T):
        T = T/365
        d1 = (math.log(S/K)+(r+(implied_vol**2)/2)*T)/(implied_vol*(np.sqrt(T)))
        delta = N(d1)
        return delta
    
    df['Delta'] = df.apply(lambda row: delta(K, r, implied_vol, row['Stock Price'], row['Days till expiry']), axis=1)

    # Calculate "Number of shares to buy" column
    df['Number of shares to buy'] = 0  # Initialize the column to zero

    # Calculate the "Number of shares to buy" column iteratively
    new_delta = 0
    for i,j in df.iterrows():
        # print(j)
        # print(total_delta)
        delta = j['Delta']
        shares_to_buy = delta - new_delta
        new_delta += shares_to_buy

        
        df.at[i, 'Number of shares to buy'] = -shares_to_buy

    # df['Option value'] = df.apply(lambda row: BS(K,row['Stock Price'],r,implied_vol,row['Days till expiry']),axis =1)
    # df['Portfolio value'] = df['Option value']+df['Number of shares to buy'].cumsum()*df['Stock Price']
    df['cost of shares'] = df['Number of shares to buy']*df['Stock Price']
    df['Net stock position'] = df['Number of shares to buy'].cumsum()
    # df['Cost of hedging'] = df['cost of shares'].cumsum()

    # last_row = pd.Series(data = [0,101,0,-1*df['Net stock position'][-1],-1*df['Net stock position'][-1]*df['Stock Price'][-1],0])
    # df
    df.loc[len(df)] = [0,last_s_price,0,-df['Net stock position'].iloc[-1],-df['Net stock position'].iloc[-1]*df['Stock Price'].iloc[-1],0]

    option_cost = BS(K,df['Stock Price'].iloc[0],r,implied_vol,df['Days till expiry'].iloc[0]/365)
    option_pnl = (max((df['Stock Price'].iloc[-1]-K),0) - option_cost)
    stock_pnl = -df['cost of shares'].sum()
    PnL = stock_pnl+option_pnl

    return option_pnl,stock_pnl,PnL







In [486]:
stock_profit = []
total_profit = []
for i in range(1000):
    a,b,c = get_dynamically_hedged_PnL(1,2)
    stock_profit.append(b)
    total_profit.append(c)



In [487]:
import plotly.express as px



# Create a histogram using Plotly Express for data1
fig = px.histogram(stock_profit, nbins=100)

# Add another histogram for data2
# fig.add_trace(px.histogram(total_profit, nbins=100).data[0])

# Customize the plot (optional)
fig.update_layout(
    title="Stock PnL",
    xaxis_title="Profit",
    yaxis_title="Frequency",
    showlegend=True
)

# Display the plot
fig



ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'bingroup': 'x',
              'hovertemplate': 'variable=0<br>value=%{x}<br>count=%{y}<extra></extra>',
              'legendgroup': '0',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '0',
              'nbinsx': 100,
              'offsetgroup': '0',
              'orientation': 'v',
              'showlegend': True,
              'type': 'histogram',
              'x': array([ 0.10676552,  2.51343116, -1.44983257, ...,  0.42364613,  1.12167315,
                           1.62390929]),
              'xaxis': 'x',
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'title': {'text': 'variable'}, 'tracegroupgap': 0},
               'margin': {'t': 60},
               'showlegend': True,
               'template': '...',
               'title': {'text': 'Stock PnL'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Profit'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Frequency'}}}
})

In [475]:
mean(stock_profit)

TypeError: 'int' object is not callable

In [ ]:
def get_statically_hedged_PnL(realised_vol,implied_vol):
    Num = 100

    day = pd.Series(range(Num,0,-1),name = 'Days till expiry')
    K = 100
    r = 0.0
    # realised_vol = 2
    # implied_vol = 1
    mean = 100

    stock_price = np.random.normal(mean,realised_vol,len(day)+1)
    last_s_price = stock_price.tolist().pop()
    stock_price = pd.Series(stock_price[:-1],index = day.index,name = 'Stock Price')

    df = pd.concat([day,stock_price],axis = 1)

    def delta(K, r, implied_vol, S, T):
        T = T/365
        d1 = (math.log(S/K)+(r+(implied_vol**2)/2)*T)/(implied_vol*(np.sqrt(T)))
        delta = N(d1)
        return delta
    
    df['Delta'] = df.apply(lambda row: delta(K, r, implied_vol, row['Stock Price'], row['Days till expiry']), axis=1)

    # Calculate "Number of shares to buy" column
    df['Number of shares to buy'] = 0  # Initialize the column to zero

    # Calculate the "Number of shares to buy" column iteratively
    new_delta = 0
    for i,j in df.iterrows():
        # print(j)
        # print(total_delta)
        delta = j['Delta']
        shares_to_buy = delta - new_delta
        new_delta += shares_to_buy

        
        df.at[i, 'Number of shares to buy'] = -shares_to_buy

    # df['Option value'] = df.apply(lambda row: BS(K,row['Stock Price'],r,implied_vol,row['Days till expiry']),axis =1)
    # df['Portfolio value'] = df['Option value']+df['Number of shares to buy'].cumsum()*df['Stock Price']
    df['cost of shares'] = df['Number of shares to buy']*df['Stock Price']
    df['Net stock position'] = df['Number of shares to buy'].cumsum()
    # df['Cost of hedging'] = df['cost of shares'].cumsum()

    # last_row = pd.Series(data = [0,101,0,-1*df['Net stock position'][-1],-1*df['Net stock position'][-1]*df['Stock Price'][-1],0])
    # df
    df.loc[len(df)] = [0,last_s_price,0,-df['Net stock position'].iloc[-1],-df['Net stock position'].iloc[-1]*df['Stock Price'].iloc[-1],0]

    option_cost = BS(K,df['Stock Price'].iloc[0],r,implied_vol,df['Days till expiry'].iloc[0]/365)
    option_pnl = (max((df['Stock Price'].iloc[-1]-K),0) - option_cost)
    stock_pnl = -df['cost of shares'].sum()
    PnL = stock_pnl+option_pnl

    return option_pnl,stock_pnl,PnL







In [22]:
np.std([102,101.6,102.53,101.3,101,99,100.9,102.7,102.6,101.6,103.7,102.7]) 

1.1569653192727951